In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("sale_end_int_all_data_0816.csv", encoding = 'utf-8')
df

,고용인원,투자유치횟수,서비스/제품 수,주소,연간 퇴사율,공동 창업자 수,총 투자자 수,특허기술 평균등급,최종투자단계,산업군,...,순이익_2020(만원),순이익_2021(만원),자산_2020(만원),자산_2021(만원),자본_2020(만원),자본_2021(만원),2022기준 시장규모(만원),최근투자유치(만원),년차,자본잠식률
0,8,1,1,인천 연수구,24,1,1,2.0,seed,전기차배터리,...,-70,14000,100,31000,100,101,25490000000,10000,3,96.54
1,13,1,3,인천 연수구,0,1,1,2.0,pre-A,자율주행차,...,-100,19000,3535,84000,2899,22000,16619999999,800000,3,-633.33
2,2,1,3,경남 창원시,100,1,1,2.0,seed,감속기,...,-17000,-23000,54000,38000,-9232,-32000,4820000000,20000,4,390.91
3,10,1,12,경기 성남시,82,1,1,2.8,seed,로봇,...,3998,1239,48000,112000,10000,18000,4260000000,10000,7,-20.00
4,11,1,1,서울 강남구,51,1,1,2.0,pre-A,디지털헬스케어,...,16000,23000,187000,585000,127000,480000,34530000000,100000,6,-3328.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,93,2,3,서울 강남구,17,1,3,0.0,M&A,메타버스,...,-180000,-396000,198000,738000,-187000,516000,8130000000,1200000,6,-12694.45
897,12,1,0,부산 금정구,73,1,1,0.0,M&A,유아신발,...,67000,44000,217000,248000,192000,217000,4870000000,500000,19,-623.33
898,136,4,1,경기 평택시,38,1,11,0.0,M&A,화물운송,...,-247000,-480000,1294000,3934000,937000,2956000,50360000000,920000,7,-13336.36
899,7,3,1,서울 서초구,100,1,6,3.0,M&A,AI 챗봇,...,17000,-714,7469,1998,-12000,-13000,710000000,30000,8,293.37


In [3]:
# 가중치 점수별 딕셔너리 만들기

weight = {('서울 강남구') : 12,
          ('서울 서초구') : 10,
          ('서울 마포구') : 9,
          ('서울 중구', '서울 성동구', '서울 구로구') : 8,
          ('서울 종로구', '서울 용산구', '서울 광진구', '서울 동대문구', '서울 중랑구',
           '서울 성북구', '서울 강북구', '서울 도봉구', '서울 노원구', '서울 은평구',
           '서울 서대문구', '서울 양천구', '서울 강서구', '서울 금천구', '서울 영등포구',
           '서울 동작구', '서울 관악구', '서울 송파구', '서울 강동구') : 7,
          ('경기 성남시') : 6,
          ('대전', '경기 수원시', '경기 용인시', '경기 안양시', '경기 안산시',
           '경기 과천시', '경기 광명시', '경기 광주시', '경기 군포시', '경기 부천시',
           '경기 시흥시', '경기 김포시', '경기 안성시', '경기 오산시', '경기 의왕시',
           '경기 이천시', '경기 평택시', '경기 하남시', '경기 화성시', '경기 여주시',
           '경기 양평군', '경기 고양시', '경기 구리시', '경기 남양주시', '경기 동두천시',
           '경기 양주시', '경기 의정부시', '경기 파주시', '경기 포천시', '경기 연천군',
           '경기 가평군') : 5,
          ('인천', '부산') : 4,
          ('충북', '충남', '경상') : 3,
          ('광주', '경북', '경남', '제주') : 2,
          ('대구', '울산', '전북', '전남') : 1}

In [12]:
def calculate_score(address):
    for keywords, score in weight.items():
        if isinstance(keywords, str): # 키워드가 문자열인 경우(키워드가 하나인 경우)
        # 즉, 서울 강남구라고 되어있는 경우임
            if keywords in address:
                return score
        else:  # 키워드가 여러 개인 경우
        # 즉, 서울 강남구 뭐시기 라고 되어있는 경우
            if any(keyword in address for keyword in keywords):
                return score
    return 0

In [13]:
df['주소 점수'] = df['주소'].apply(calculate_score)

In [14]:
df['주소 점수']

0       4
1       4
2       2
3       6
4      12
       ..
896    12
897     4
898     5
899    10
900     3
Name: 주소 점수, Length: 901, dtype: int64

In [15]:
# 기존 주소와 산업군 피쳐 제거
df.drop(['주소', '산업군'], axis = 1, inplace = True)

In [ ]:
df.to_csv("fix_sale_final_ml_data_0816.csv", encoding = 'utf-8', index = False)